In [51]:
import pandas as pd
import numpy as np
from nba_help import tracking, pbp, plotting, get_basic_statistics
import matplotlib.pyplot as plt

In [52]:
def get_score_assist_df(scorers, assistor, pass_defender, shot_defender):
    scorers_df = pd.read_csv(scorers, header=None)
    assistors_df = pd.read_csv(assistor, header=None)
    pass_defenders_df = pd.read_csv(pass_defender, header=None)
    shot_defenders_df = pd.read_csv(shot_defender, header=None)
    score_assist_defend_df = pd.concat([assistors_df, scorers_df, pass_defenders_df, shot_defenders_df], axis=1)
    
    score_assist_defend_df.columns = ["AssistorName", 
                                      "AssistorX", 
                                      "AssistorY", 
                                      "ScorerName", 
                                      "ScorerX", 
                                      "ScorerY", 
                                      "PassDefenderName", 
                                      "PassDefenderX", 
                                      "PassDefenderY",
                                      "ShotDefenderName", 
                                      "ShotDefenderX", 
                                      "ShotDefenderY"]
    
    score_assist_defend_df = score_assist_defend_df.astype({'AssistorX': float, 
                                                            'AssistorY': float, 
                                                            'ScorerX': float, 
                                                            'ScorerY': float,
                                                            'ShotDefenderX': float,
                                                            'ShotDefenderY': float,
                                                            'PassDefenderX': float,
                                                            'PassDefenderY': float})
    
    for i, row in score_assist_defend_df.iterrows():
        if (row['AssistorX']) >= 47.0:
            score_assist_defend_df.at[i, 'AssistorX'] = 94-(row['AssistorX'])
            score_assist_defend_df.at[i, 'AssistorY'] = 50-(row['AssistorY'])
        if float(row['ScorerX']) >= 47.0:
            score_assist_defend_df.at[i, 'ScorerX'] = 94-(row['ScorerX'])
            score_assist_defend_df.at[i, 'ScorerY'] = 50-(row['ScorerY'])
        if float(row['ShotDefenderX']) >= 47.0:
            score_assist_defend_df.at[i, 'ShotDefenderX'] = 94-(row['ShotDefenderX'])
            score_assist_defend_df.at[i, 'ShotDefenderX'] = 50-(row['ShotDefenderX'])
        if float(row['PassDefenderX']) >= 47.0:
            score_assist_defend_df.at[i, 'PassDefenderX'] = 94-(row['PassDefenderX'])
            score_assist_defend_df.at[i, 'PassDefenderX'] = 50-(row['PassDefenderX'])
            
    score_assist_defend_df['TeamName'] = ""
    for i, row in score_assist_defend_df.iterrows():
        assistor = get_basic_statistics.get_basic_stats_by_player(season="2015-16", player_id=str(int(row['AssistorName'])))
        shooter = get_basic_statistics.get_basic_stats_by_player(season="2015-16", player_id=str(int(row['ScorerName'])))
        shot_defender = get_basic_statistics.get_basic_stats_by_player(season="2015-16", player_id=str(int(row['ShotDefenderName'])))
        pass_defender = get_basic_statistics.get_basic_stats_by_player(season="2015-16", player_id=str(int(row['PassDefenderName'])))

        if shooter['TEAM_ABBREVIATION'] == assistor['TEAM_ABBREVIATION']:
            score_assist_defend_df.at[i, 'TeamName'] = shooter['TEAM_ABBREVIATION']
            
        if shooter['TEAM_ABBREVIATION'] != pass_defender['TEAM_ABBREVIATION'] and shooter['TEAM_ABBREVIATION'] != shot_defender['TEAM_ABBREVIATION']:
            score_assist_defend_df.at[i, 'OtherTeamName'] = pass_defender['TEAM_ABBREVIATION']
            
    return score_assist_defend_df

In [53]:
score_assist_defend_make_df = get_score_assist_df("nonaggregated/scorers_on_assist_new_algo.csv", 
                                                  "nonaggregated/assistor_on_assist_new_algo.csv", 
                                                  "nonaggregated/defender_on_assist_new_algo.csv",
                                                  "nonaggregated/shoot_defender_on_assist_new_algo.csv")

In [54]:
score_assist_defend_miss_df = get_score_assist_df("nonaggregated/scorers_on_miss_new_algo.csv", 
                                                  "nonaggregated/assistor_on_miss_new_algo.csv", 
                                                  "nonaggregated/defender_on_miss_new_algo.csv",
                                                  "nonaggregated/shoot_defender_on_miss_new_algo.csv")

In [55]:
def is_a_three(x, y):
    x_line = 300-((y-250)**2)/188 
    return x > x_line

In [56]:
def get_location_on_court(x, y):
    location = ""
    if x < 130:
        location += "Corner"
        if y >= 475 or y <= 25:
            location += "3"
        elif (25 < y < 175) or (325 < y < 475):
            location += "2"
        else:
            location += "Paint"
    else:
        three = is_a_three(x,y)            
        if three:
            location += "AboveTheBreak3"
        else:
            location += "Midrange2"
            
    if location == "CornerPaint":
        location = "Paint"
        
    return location

In [57]:
def get_location_for_df(df):
    df['LocationScorer'] = ""
    df['LocationAssistor'] = ""
    for i, row in df.iterrows():
        df.at[i, 'LocationScorer'] = get_location_on_court(row['ScorerX']*10, row['ScorerY']*10)
        df.at[i, 'LocationAssistor'] = get_location_on_court(row['AssistorX']*10, row['AssistorY']*10)

In [58]:
get_location_for_df(score_assist_defend_make_df)
get_location_for_df(score_assist_defend_miss_df)

In [59]:
score_assist_defend_make_df = score_assist_defend_make_df.loc[score_assist_defend_make_df['TeamName'].isnull()==False]  
score_assist_defend_miss_df = score_assist_defend_miss_df.loc[score_assist_defend_miss_df['TeamName'].isnull()==False]  

In [60]:
score_assist_defend_make_df.to_csv("aggregated/assist_new_algo.csv")
score_assist_defend_miss_df.to_csv("aggregated/miss_new_algo.csv")